#### bibliotecas necessárias
- pandas==2.2.2
- llama-index==0.11.20
- llama-index-llms-groq==0.2.0
- llama-index-experimental==0.4.0
- gradio==5.4.0
- fpdf==1.7.2

pip install llama-index==0.11.20 && pip install llama-index-llms-groq==0.2.0 && pip install llama-index-experimental==0.4.0 && pip install gradio==5.4.0 && pip install fpdf==1.7.2

- OBS: Também é necessário uma key atravez do https://groq.com, sinalizado na variável "groq_key" na celula abaixo.
- OBS2: Bem como um arquivo para análise, no caso, para este projeto utilizamos as informações de uma tabela fictícia da Valquíria Alencar, instrutora do curso de LLM LlamaIndex na Alura

In [ ]:
import pandas as pd
import textwrap
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.experimental.query_engine import PandasQueryEngine
groq_key = ''
url = 'https://github.com/alura-cursos/llamaIndex_pandas_query/blob/main/Dados/vendas.csv'

Settings.llm = Groq(model='llama3-70b-8192', api_key=groq_key)

In [ ]:
DF = pd.read_csv(url)

In [ ]:
# A FORMA MAIS DIRETA DE RESPOSTAS UTILIZANDO QUERY_ENGINE
query_engine = PandasQueryEngine(df=DF, verbose=True)

response = query_engine.query('Qual a forma de pagamento mais utilizada?')

In [ ]:
# RESPOSTAS MAIS COMPLETAS ALÉM DO VALOR FINAL, UMA EXPLICAÇÃO
# TEXTWRAP É APENAS PARA FACILITAR A VISUALIZAÇÃO, NÃO É NECESSÁRIO
query_engine = PandasQueryEngine(df=DF, verbose=True, synthesize_response=True)

response = query_engine.query('Qual a média de avaliação para cada filial?')
print(textwrap.fill(response.response, width=100))

In [ ]:
query_engine = PandasQueryEngine(df=DF, verbose=True, synthesize_response=True)

response = query_engine.query('')
